# Задача классификации текста

В этом модуле мы начнем с простой задачи классификации текста на основе набора данных **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: наша задача будет состоять в классификации заголовков новостей по одной из 4 категорий: мир, спорт, бизнес и Sci/Tech. 

## Набор данных

Для загрузки набора данных мы будем использовать **[TensorFlow Datasets API](https://www.tensorflow.org/datasets)**.

In [5]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

dataset = tfds.load('ag_news_subset')

Теперь мы можем получить доступ к обучающей и тестовой частям набора данных, используя `dataset['train']` и `dataset['test']` соответственно:

In [6]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Распечатаем первые 10 заголовков из нашего набора данных: 

In [7]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Векторизация текста

Теперь нам нужно преобразовать текст в **векторное представление**, т.е. представить его в виде тензоров, которые можно подать на вход нейросети. Это называется **векторизацией**.

Можно использовать два вида представлений: 
* На уровне символов - на вход нейросети подаются отдельные буквы
* На уровне слов - на вход нейросети подаются целиком слова

В обоих случаях используется одинаковый подход:

* Используется **токенизатор** для разделения текста на **токены** (это могут быть либо отдельные символы и знаки препинания, либо отдельные слова)
* Создается **словарь** из этих токенов.

Оба этих шага можно выполнить с помощью слоя **TextVectorization**. Сначала создается экземпляр объекта векторизатора, а затем необходимо вызвать метод `adapt`, чтобы просмотреть весь текст и заполнить словарь.

### Ограничение размера словаря

В примере с набором данных AG News размер словаря довольно большой, более 100 тысяч слов. Вообще говоря, нам не нужны слова, которые редко присутствуют в тексте — они будут содержаться только в нескольких предложениях, и модель не будет эффективно использовать их при обучении. Таким образом, имеет смысл ограничить размер словаря меньшим числом, передав конструктору векторизатора соответствующий аргумент.

In [8]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(15000).map(lambda x: x['title']+' '+x['description']))

> **Примечание**, что мы используем только подмножество всего набора данных для создания словарного запаса, чтобы ускорить время выполнения. При этом мы рискуем, что некоторые слова из всего датасета не будут включены в словарь - при этом они будут проигнорированы во время обучения. Таким образом, использование всего словарного запаса и прохождение всего набора данных для построения словаря может повысить конечную точность, но скорее всего незначительно.

Теперь мы можем получить доступ к фактическому словарю:

In [9]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'of', 'in', 'and', 'on', 'for']
Length of vocabulary: 36705


С помощью векторизатора мы можем легко закодировать любой текст в набор чисел:

In [10]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 290, 1991,    3,  317,   12, 1076, 1811], dtype=int64)>

## Мешок слов (Bag-of-Words, BoW)

Поскольку многие слова несут на себе некоторое конкретное значение, иногда мы можем выяснить общий смысл фрагмента текста, просто взглянув на отдельные имеющиеся в нем слова, независимо от их порядка в предложении. Например, при классификации новостей такие слова, как *погода* и *снег*, скорее всего, будут указывать *прогноз погоды*, в то время как такие слова, как *акции* и *доллар*, будут засчитываться в *финансовые новости*.

Представление **мешок слов** (*Bag-of-words*, BoW) - это наиболее простое векторное представление текста. Мы видели ранее, что каждое слово в тексте связывается с некоторым индексом в словаре. Текст представляется вектором размерностью с количество слов в словаре, и $i$-й элемент содержит количество вхождений слова с индексом $i$ в документ.

![Изображение, показывающее, как в памяти представлено векторное представление мешка слов.](images/bag-of-words-example.png) 

> **Примечание**: Вы также можете думать о BoW как о сумме всех one-hot-encoded векторов индексов отдельных слов в тексте.

Ниже приведен пример того, как создать представление мешка слов с помощью библиотеки Scikit Learn:

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Мы также можем использовать векторизатор Keras, определённый нами выше, преобразуя каждое слово в one-hot-encoding и складывая:

In [12]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

> **Примечание**: Результат отличается от предыдущего примера, поскольку в случае с векторизатором Keras длина вектора соответствует размеру словаря, который был построен из всего набора данных AG News, в то время как в примере Scikit Learn мы построили словарь из небольшого фрагмента текста на лету. 

## Обучение классификатора BoW

Теперь, когда мы узнали, как построить представление нашего текста в виде мешка слов, давайте обучим классификатор, который его использует. Во-первых, нам нужно преобразовать наш набор данных в представление мешка слов. Этого можно достичь, используя функцию `map` следующим образом:

In [13]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Теперь определим простой нейросетевой классификатор с одним линейным слоем. Входной размер — `vocab_size`, а выходной размер соответствует количеству классов (4). Поскольку мы решаем задачу классификации, конечной функцией активации является **softmax**:

In [15]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

  1/938 [..............................] - ETA: 11:44 - loss: 1.3802 - acc: 0.2812

KeyboardInterrupt: 

Поскольку у нас 4 класса, точность выше 80% является хорошим результатом.

## Обучение классификатора как одной сети

Поскольку векторизатор также является слоем Keras, мы можем определить сеть, которая включает его как первый слой, и обучать такую сеть, подавая ей на вход текст. Таким образом, нам не нужно предварительно векторизировать набор данных с помощью `map`, мы можем просто передать исходный набор данных на вход сети.

> **Примечание**: Нам все равно придется применять `map` к нашему набору данных для преобразования полей в исходном датасете (таких как `title`, `description` и `label`) в кортежи. Однако при использовании сторонних данных мы можем сразу построить набор данных с требуемой структурой.

In [16]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
tf.one_hot (TFOpLambda)      (None, None, 36705)       0         
_________________________________________________________________
tf.math.reduce_sum (TFOpLamb (None, 36705)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 146824    
Total params: 146,824
Trainable params: 146,824
Non-trainable params: 0
_________________________________________________________________
 17/938 [..............................] - ETA: 21:42 - loss: 1.3663 - acc: 0.3755

KeyboardInterrupt: 

## Биграммы, триграммы и n-граммы

Одним из ограничений подхода BoW является то, что некоторые слова являются частью многословных выражений, например, слово hot dog (хот-дог) имеет совершенно иное значение, чем слова hot (горячий) и dog (собака) в других контекстах. Если мы представляем слова «горячий» и «собака» используя для всех контекстов одни и те же векторы, то наша модель BoW не сможет отличать разницу в смыслах.

> Более сложные нейросетевые модели не страдают таким недостатком, поскольку, как и свёрточные сети, они могут вычленять паттерны в последовательностях слов. Но в BoW порядок слов вообще никак не учитывается.

Чтобы решить эту проблему, совместно с BoW часто используются **биграммы**, т.е. последовательности из двух соседних слов. Для обучения классификторов имеют значения частота каждого слова в отдельности, а также биграмм, триграмм и т.д. В биграммовом представлении мы добавляем в словарь все пары слов, в дополнение к оригинальным словам.

Ниже приведен пример того, как создать мешок биграмм с помощью Scikit Learn:

In [17]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основным недостатком n-граммового подхода является то, что размер словаря начинает расти чрезвычайно быстро. На практике нам нужно объединить n-граммовое представление с техникой уменьшения размерности, такой как *embeddings*, о которых мы поговорим в следующем блоке.

Чтобы использовать n-граммовое представление в нашем наборе данных **AG News**, нам нужно передать параметр `ngrams` конструктору `TextVectorization`. Длина биграммового словаря получается **значительно больше**, в нашем случае это более 1,3 миллиона токенов! Имеет смысл ограничить количество биграмм-токенов каким-то разумным числом.

Код выше для обучения классификатора может быть использовать почти без изменений, однако это оказывается очень неэффективным для памяти. В следующем блоке мы будем обучать биграмм-классификатор с помощью эмбеддингов. А пока вы можете поэкспериментировать с обучением биграмм-классификатора в этом блокноте и посмотреть, сможете ли вы получить более высокую точность.

## Автоматическое вычисление векторов BoW

В приведенном выше примере мы рассчитали векторы BoW вручную, суммируя one-hot-encodings отдельных слов. Однако последняя версия TensorFlow позволяет автоматически вычислять векторы BoW, передавая параметр `output_mode='count'` конструктору векторизатора. Это значительно упрощает определение и обучение нашей модели:

In [18]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(15000).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
126/938 [===>..........................] - ETA: 14s - loss: 1.0657 - acc: 0.7289

KeyboardInterrupt: 

Поскольку наша модель - однослойная, то мы можем интерпретировать значения весов слоя классификатора:

In [ ]:
w = model.layers[1].weights[0]
w.shape

Посмотрим на топ-10 самых важных слов для каждой из новостных категорий. Для этого используем `argsort` чтобы получить индексы самых больших весов в строке, и затем возьмём слова из словаря в соответствюущей позиции:

In [ ]:
import pandas as pd

vocab = model.layers[0].get_vocabulary()

pd.DataFrame( { classes[k] : [ vocab[i] for i in np.argsort(w[:,k])[::-1][:20]] for k in range(4) })

## Term frequency - inverse document frequency (TF-IDF)

В представлении BoW частота вхождений слов учитывается независимо от самого слова. Однако можно заметить, что частые слова, такие как *a* и *in*, гораздо менее важны для классификации, чем специализированные термины. 

**TF-IDF** расшифровывается как **term frequency - inverse document frequency**. Это разновидность BoW, где количество вхождений слов в документе нормируется с учётом частоты встречаемости слова в корпусе.

Более формально вес $w_{ij}$ слова $i$ в документе $j$ определяется как:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
где
* $tf_{ij}$ — количество вхождений $i$ в $j$, т.е. значение BoW, которое мы видели ранее
* $N$ - количество документов в коллекции
* $df_i$ - количество документов во всей коллекции, содержащих слово $i$

Значение TF-IDF $w_{ij}$ увеличивается пропорционально количеству появлений слова в документе и уменьшается пропорционально количеству документов в корпусе, содержащем слово, что помогает скорректировать тот факт, что некоторые слова появляются чаще, чем другие. Например, если слово появляется в *каждом* документе в коллекции, $df_i=N$, и $w_{ij}=0$, т.е. эти термины будут полностью проигнорированы.

Вы можете легко создать tf-IDF векторизацию текста с помощью Scikit Learn:

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

В Keras слой `TextVectorization` может автоматически вычислять частоты TF-IDF - нам лишь нужно передать параметр `output_mode='tf-idf`: 

In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Заключение 

Несмотря на то, что представление TF-IDF позволяет учитывать действительно важные слова, оно по-прежнему не учитывает порядок слов и, следовательно, контекст. Как сказал в 1935 году известный лингвист Дж.Р. Ферт: «Полное значение слова всегда контекстуально, и изучение значения, не учитывающее контекст, не может восприниматься всерьез». Мы узнаем, как собирать контекстную информацию из текста с помощью языкового моделирования позже в курсе.